In [1]:
import requests
url = "http://183.82.112.181:3402/Trans/CurrentVoltage/868997035786613/all"
user = "admin"
passwd = "admin@123"
auth_values = (user, passwd)
response = requests.get(url, auth=auth_values)
print(response.json())

ConnectionError: HTTPConnectionPool(host='183.82.112.181', port=3402): Max retries exceeded with url: /Trans/CurrentVoltage/868997035786613/all (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000221AB78D220>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
import pandas as pd
lis=response.json()
head=list(lis[0].keys())
head
rawdata=[]
for i in range(0, len(lis)):
    l=[]
    for x in head:
        if(x=='DeviceImei' or x=='A_id'):
            l.append(lis[i][x])
        elif(x=='DeviceTimeStamp'):
            s=lis[i][x]
            l.append(s[0:16])
        else:
            l.append(float(lis[i][x]))
    rawdata.append(l)
    #rint(lis[0][x], type(lis[0][x]))
def Reverse(lst): 
    return [ele for ele in reversed(lst)] 
data=Reverse(rawdata)
print(data)

In [ ]:
cv = pd.DataFrame(data, columns=head)
cv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

In [ ]:
plot_data = [
    go.Scatter(
        x=cv['DeviceTimeStamp'],
        y=cv['IL1'],
        name='IL1'
    ),
    go.Scatter(
        x=cv['DeviceTimeStamp'],
        y=cv['IL2'],
        name='IL2'
    ),
    go.Scatter(
        x=cv['DeviceTimeStamp'],
        y=cv['IL3'],
        name='IL3'
    )
    
]
plot_layout = go.Layout(
        title='Load'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
cv['DeviceTimeStamp'] = pd.to_datetime(cv['DeviceTimeStamp'])
cv

In [ ]:
import datetime
x=cv['DeviceTimeStamp'].max()
y=cv['DeviceTimeStamp'].min()
days = datetime.timedelta(7)
week=x-days

In [ ]:
il1nxtweek=cv[cv['DeviceTimeStamp']>=week]
il1nxtweek

In [ ]:
plot_data = [
    go.Scatter(
        x=il1nxtweek['DeviceTimeStamp'],
        y=il1nxtweek['IL1'],
        name='IL1'
    ),
    go.Scatter(
        x=il1nxtweek['DeviceTimeStamp'],
        y=il1nxtweek['IL2'],
        name='IL2'
    ),
    go.Scatter(
        x=il1nxtweek['DeviceTimeStamp'],
        y=il1nxtweek['IL3'],
        name='IL3'
    )
    
]
plot_layout = go.Layout(
        title='Load'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
il1nxtweek_high=il1nxtweek.query('IL1>=1.2*180', inplace=False)
il1nxtweek_high

In [ ]:
total_il1nxtwk_high=len(il1nxtweek_high)

In [ ]:
cv_il1_high=cv.query('IL1>=1.2*180', inplace=False)
cv_il1_high

In [ ]:
total_il1_high=len(cv_il1_high)

In [ ]:
plot_data = [
    go.Scatter(
        x=cv['DeviceTimeStamp'],
        y=cv['IL1'],
        name='IL1'
    ),
    go.Scatter(
        x=cv_il1_high['DeviceTimeStamp'],
        y=cv_il1_high['IL1'],
        mode='markers',
        name='Alarm',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'red',
            opacity= 0.8
           )
    )
    
]
plot_layout = go.Layout(
        title='IL1 (All time), Alarm count = '+str(total_il1_high)
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
plot_data = [
    go.Scatter(
        x=il1nxtweek['DeviceTimeStamp'],
        y=il1nxtweek['IL1'],
        name='IL1'
    ),
    go.Scatter(
        x=il1nxtweek_high['DeviceTimeStamp'],
        y=il1nxtweek_high['IL1'],
        mode='markers',
        name='Alarm',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'red',
            opacity= 0.8
           )
    )
    
]
plot_layout = go.Layout(
        title='IL1 (All time), Alarm count last week = '+str(total_il1nxtwk_high)
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [ ]:
#Arima
from pandas.plotting import autocorrelation_plot

In [ ]:
dataset_il1 = il1nxtweek['IL1'].values
dataset_il1 = dataset_il1.astype('float32')

In [ ]:
autocorrelation_plot(il1nxtweek['IL1'])

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
# fit model
model_il1 = ARIMA(il1nxtweek['IL1'], order=(5,1,0))
model_fit_il1 = model_il1.fit(disp=0)
print(model_fit_il1.summary())

In [ ]:
# plot residual errors
from pandas import DataFrame
residuals = DataFrame(model_fit_il1.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
X = il1nxtweek['IL1'].values

In [ ]:
size = int(len(X) * 6/7)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
confidence=list()
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit(disp=0)
	output, error, conf = model_fit.forecast()
	yhat = output[0]
	predictions.append([yhat, conf[0][0], conf[0][1]])
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f,' % (yhat, obs))
'''error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()'''

In [ ]:
pred=pd.DataFrame(predictions)

In [ ]:
error = mean_squared_error(test, pred[0])
print('Test MSE: %.3f' % error)
rmse_il1 = np.sqrt(mean_squared_error(test, pred[0]))
print('Test RMSE: %.3f' % rmse_il1)

In [ ]:
from matplotlib import pyplot
#pyplot.plot(train)
pyplot.plot(test)
pyplot.plot(pred[0], color='red')
pyplot.show()

In [ ]:
x_data_il1=pd.DataFrame(il1nxtweek['IL1'])
x_data_il1

In [ ]:
test1=test.reshape(-1, 1)

In [ ]:
test1=pd.DataFrame(test1)
test1

In [ ]:
predictions1=pd.DataFrame(pred[0])

In [ ]:
predictions1

In [ ]:
low=pd.DataFrame(pred[1])

In [ ]:
low

In [ ]:
up=pd.DataFrame(pred[2])
up

In [ ]:
pred[0]

In [ ]:
train1=pd.DataFrame(train)

In [ ]:
plt.plot(train1)

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(x_data_il1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[0:len(train), :] = train1

In [ ]:
testPredictPlot = np.empty_like(x_data_il1)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train1):len(x_data_il1), :] = predictions1

In [ ]:
lower = np.empty_like(x_data_il1)
lower[:, :] = np.nan
lower[len(train1):len(x_data_il1), :] = low

In [ ]:
upper = np.empty_like(x_data_il1)
upper[:, :] = np.nan
upper[len(train1):len(x_data_il1), :] = up

In [ ]:
plt.figure(figsize=(17,8))
#plt.plot(x_data, label="original")
plt.title("ARIMA")
plt.xlabel("time")
plt.ylabel("IL2")
plt.plot(trainPredictPlot)
plt.plot(upper, color="Grey")
plt.plot(lower, color="Grey")
plt.plot(testPredictPlot, color='Red')
plt.show()
#BLue is original
#Orange is training
#Red is Predicted

In [ ]:
testpp=pd.DataFrame(testPredictPlot)

In [ ]:
plt.plot(testpp)

In [ ]:
trainpp=pd.DataFrame(trainPredictPlot)

In [ ]:
plt.plot(trainpp)

In [ ]:
df2=pd.DataFrame(il1nxtweek['DeviceTimeStamp'])

In [ ]:
df2=df2.reset_index()
df2

In [ ]:
df2=df2.drop(['index'], axis = 1)

In [ ]:
training = df2.merge(trainpp, left_index=True, right_index=True)

In [ ]:
predicted = df2.merge(testpp, left_index=True, right_index=True)
predicted

In [ ]:
original1_il1=il1nxtweek

In [ ]:
original1_il1 = original1_il1.set_index('DeviceTimeStamp')

In [ ]:
plt.plot(original1_il1['IL1'])

In [ ]:
trn_il1=training

In [ ]:
trn_il1 = trn_il1.set_index('DeviceTimeStamp')

In [ ]:
plt.plot(trn_il1)

In [ ]:
prd_il1=predicted

In [ ]:
prd_il1 = prd_il1.set_index('DeviceTimeStamp')

In [ ]:
prd_il1

In [ ]:
fault_pred_il1=prd_il1[prd_il1[0]>1.2*180]
fault_pred_il1

In [ ]:
il1_faults_next_day=len(fault_pred_il1)

In [ ]:
s_il1=''
if(il1_faults_next_day>0):
    s_il1='Fault may occur next 24hrs'
else:
    s_il1='Fault may not occur next 24hrs'

In [ ]:
upb=pd.DataFrame(upper)
ub_il1 = df2.merge(upb, left_index=True, right_index=True)
ub_il1= ub_il1.set_index('DeviceTimeStamp')
ub_il1

In [ ]:
lwb=pd.DataFrame(lower)
lb_il1 = df2.merge(lwb, left_index=True, right_index=True)
lb_il1= lb_il1.set_index('DeviceTimeStamp')
lb_il1

In [ ]:
plt.figure(figsize=(17,8))

plt.title("ARIMA")
plt.xlabel("time")
plt.ylabel("IL1")
plt.plot(original1_il1['IL1'], label='Original')
plt.plot(trn_il1, label='Training')
plt.plot(ub_il1, color='Grey', label='Upper Bound' )
plt.plot(lb_il1, color='Grey', label='Lower Bound' )
plt.plot(prd_il1, color='Red', label='Predicted')

In [ ]:
plot_data = [
    go.Scatter(
        x=original1_il1.index,
        y=original1_il1['IL1'],
        name='IL1'
    ),
    go.Scatter(
        x=prd_il1.index,
        y=prd_il1[0],
        name='Predicted'
    ),
    go.Scatter(
        x=ub_il1.index,
        y=ub_il1[0],
        name='Upper Bound'
    ),
    go.Scatter(
        x=trn_il1.index,
        y=trn_il1[0],
        name='Training'
    ),
    go.Scatter(
        x=lb_il1.index,
        y=lb_il1[0],
        name='Lower Bound'
    )
    
    
]
plot_layout = go.Layout(
        title='Load, rmse='+str(rmse_il1)+', '+s_il1,
        yaxis_title='IL1',
        xaxis_title='Time'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)